# Revenue via in-app advertisement
This notebook demonstrates the profitability of in-app advertisement for our free-to-download applications on Google Play and the App Store.

In [1]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row
        
        if rows_and_columns:
            print('Number of rows:', len(dataset))
            print('Number of columns;', len(dataset[0]))
            

In [2]:
from csv import reader

In [3]:
afile = open('AppleStore.csv')
gfile = open('googleplaystore.csv')
aread = reader(afile)
gread = reader(gfile)
appleDS = list(aread)
googleDS = list(gread)

# Exploring the data to see how we can clean and use it

To help identify which apps/games are gaining a large audience we can use the following columns.

| Column Name | Description |
|-------------|-------------|
| Size_bytes | does size of app impact preferences |
| price | Allows us to find the free apps as this is all we make |
| rating_count_tot|how many people have rated the app/game |
|prime_genre| if we want to look at grouping data by category|
|user_rating| to help us identify the highest rated apps/games|



In [4]:
explore_data(appleDS,0,1,True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


Number of rows: 7198
Number of columns; 16


In [5]:
explore_data(googleDS,0,1,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows: 10842
Number of columns; 13


In [6]:
explore_data(googleDS,10472,10474)

['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']




In [7]:
apple_cols = appleDS[0]
google_cols = googleDS[0]

# Data cleaning

In [8]:
# This data was wrong. Lets remove it
# del googleDS[10473]

#### Find duplicate applications by name.
There are cases where we have duplicate applications listed, the main difference between them being the number of user ratings. We will keep the row with the highest user rating and delete the others

In [9]:
# Example showing Facebook is duplicated
for each in googleDS:
    if each[0] == 'Facebook':
        print(each,'\n')

['Facebook', 'SOCIAL', '4.1', '78158306', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device'] 

['Facebook', 'SOCIAL', '4.1', '78128208', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'August 3, 2018', 'Varies with device', 'Varies with device'] 



In [10]:
duplicates = {}
unique = []


for app in googleDS[1:]:
    name = app[0]
    reviews = app[3]
    if name in unique:
        if name in duplicates:
            if float(reviews) > duplicates[name]: #new highest rating
                duplicates[name] = float(reviews)

        else: #add to dictionary
            duplicates[name] = float(reviews)
    else:
        unique.append(name)

In [11]:
apple_duplicates = {}
apple_unique = []


for app in appleDS[1:]:
    name = app[1]
    reviews = app[5]
    if name in apple_unique:
        if name in apple_duplicates:
            if float(reviews) > apple_duplicates[name]: #new highest rating
                apple_duplicates[name] = float(reviews)

        else: #add to dictionary
            apple_duplicates[name] = float(reviews)
    else:
        apple_unique.append(name)

In [12]:
print('Number of google duplicate apps: ', len(duplicates))
print('\n')
print('Exmaples of google duplicate apps: ')
for each in sorted(duplicates.keys())[:10]:
    print(each)


Number of google duplicate apps:  798


Exmaples of google duplicate apps: 
10 Best Foods for You
1800 Contacts - Lens Store
2017 EMRA Antibiotic Guide
21-Day Meditation Experience
365Scores - Live Scores
420 BZ Budeze Delivery
8 Ball Pool
8fit Workouts & Meal Planner
95Live -SG#1 Live Streaming App
A Manual of Acupuncture


In [13]:
print('Number of apple duplicate apps: ', len(apple_duplicates))
print('\n')
print('Exmaples of apple duplicate apps: ')
for each in sorted(apple_duplicates.keys())[:10]:
    print(each)


Number of apple duplicate apps:  2


Exmaples of apple duplicate apps: 
Mannequin Challenge
VR Roller Coaster


In [14]:
cnt = 0
keys = duplicates.keys()
for each in googleDS[1:]:
    if each[0] in keys:
        cnt +=1

print('Total number of google rows duplicated',cnt)
print('Total number of google rows to be deleted', cnt-len(duplicates))
print('Total number of googlerows to remain', len(googleDS) - (cnt-len(duplicates)) -1) #-1 for header

Total number of google rows duplicated 1979
Total number of google rows to be deleted 1181
Total number of googlerows to remain 9660


In [15]:
cnt = 0
keys = apple_duplicates.keys()
for each in appleDS[1:]:
    if each[0] in keys:
        cnt +=1

print('Total number of apple rows duplicated',cnt)
print('Total number of apple rows to be deleted', cnt-len(apple_duplicates))
print('Total number of apple rows to remain', len(appleDS) - (cnt-len(apple_duplicates)) -1) #-1 for header

Total number of apple rows duplicated 0
Total number of apple rows to be deleted -2
Total number of apple rows to remain 7199


In [16]:
google_clean = []
already_added = []

for each in googleDS[1:]:
    name = each[0]
    n_reviews = float(each[2])
    
    if name not in already_added and name not in duplicates.keys():
        google_clean.append(each)
        already_added.append(name)
    elif name not in already_added and name in duplicates.keys():
        if n_reviews == duplicates[name]:
            google_clean.append(each)
            already_added.append(name)
    

In [17]:
len(google_clean)

8862

In [18]:
apple_clean = []
apple_already_added = []

for each in appleDS[1:]:
    name = each[1]
    n_reviews = float(each[5])
    
    if name not in apple_already_added and name not in apple_duplicates.keys():
        apple_clean.append(each)
        apple_already_added.append(name)
    elif name not in apple_already_added and name in apple_duplicates.keys():
        if n_reviews == apple_duplicates[name]:
            apple_clean.append(each)
            apple_already_added.append(name)

### Focus on english market
We are not interest in apps that are focused on non-english speaking markets so we will remove them from the app list.

In [19]:
def engApp(appname):
    counter = 0
    for char in appname:
        if ord(char) > 127:
            counter += 1
        if counter >= 4:
            return False
    return True

In [20]:
google_clean2 = []
for items in google_clean:
    if engApp(items[0]) == True:
        google_clean2.append(items)

In [21]:
len(google_clean2)

8817

In [22]:
apple_clean2 = []
for items in appleDS:
    if engApp(items[0]) == True:
        apple_clean2.append(items)

In [23]:
len(apple_clean2)

7198

## Remove paid for apps.
We are only interested in free apps, as this is what our market competition is.

In [24]:
google_final = []
for item in google_clean2:
    if item[6] == 'Free':
        google_final.append(item)

        apple_final = []
for item in apple_clean2:
    if item[4] == '0.0':
        apple_final.append(item)

In [25]:
print(len(google_final), len(apple_final))

8105 4056


## Profiling the apps
We are looking for apps which are successful in both apple store and goole play. We'll also break this down by genre to allow us to analyse the markets in each genre.

In [26]:
google_final[0]

['Photo Editor & Candy Camera & Grid & ScrapBook',
 'ART_AND_DESIGN',
 '4.1',
 '159',
 '19M',
 '10,000+',
 'Free',
 '0',
 'Everyone',
 'Art & Design',
 'January 7, 2018',
 '1.0.0',
 '4.0.3 and up']

In [27]:
apple_final[0]

['284882215',
 'Facebook',
 '389879808',
 'USD',
 '0.0',
 '2974676',
 '212',
 '3.5',
 '3.5',
 '95.0',
 '4+',
 'Social Networking',
 '37',
 '1',
 '29',
 '1']

In [28]:
# Function to allow user to pass a list of lists to dataset, and column number 
# to the index value. function loops through lists and pulls out the values
# based on the index, to put into a new local dictionary and returns it.

def freq_table(dataset, index):
    freq = {}
    for each in dataset:
        if each[index] in freq:
            freq[each[index]] += 1
        else:
            freq[each[index]] = 1
    
    return freq

def freq_table_percentage(dataset, index):
    freq = {}
    for each in dataset:
        if each[index] in freq:
            freq[each[index]] += 1
        else:
            freq[each[index]] = 1
    
    total = sum(freq.values())
    for k, v in freq.items():
        freq[k] = round((v/total) * 100,2)
    
    return freq


In [29]:
#apple_freq = freq_table_percentage(apple_final,11)
apple_freq = freq_table(apple_final,11)

In [30]:
apple_freq

{'Book': 66,
 'Business': 20,
 'Catalogs': 9,
 'Education': 132,
 'Entertainment': 334,
 'Finance': 84,
 'Food & Drink': 43,
 'Games': 2257,
 'Health & Fitness': 76,
 'Lifestyle': 94,
 'Medical': 8,
 'Music': 67,
 'Navigation': 20,
 'News': 58,
 'Photo & Video': 167,
 'Productivity': 62,
 'Reference': 20,
 'Shopping': 121,
 'Social Networking': 143,
 'Sports': 79,
 'Travel': 56,
 'Utilities': 109,
 'Weather': 31}

In [31]:
google_freq = freq_table_percentage(google_final,1)

In [32]:
google_freq

{'ART_AND_DESIGN': 0.69,
 'AUTO_AND_VEHICLES': 1.01,
 'BEAUTY': 0.65,
 'BOOKS_AND_REFERENCE': 2.23,
 'BUSINESS': 4.63,
 'COMICS': 0.63,
 'COMMUNICATION': 3.01,
 'DATING': 1.38,
 'EDUCATION': 1.0,
 'ENTERTAINMENT': 0.7,
 'EVENTS': 0.78,
 'FAMILY': 19.7,
 'FINANCE': 3.84,
 'FOOD_AND_DRINK': 1.18,
 'GAME': 9.48,
 'HEALTH_AND_FITNESS': 2.86,
 'HOUSE_AND_HOME': 0.75,
 'LIBRARIES_AND_DEMO': 1.01,
 'LIFESTYLE': 4.12,
 'MAPS_AND_NAVIGATION': 1.46,
 'MEDICAL': 3.41,
 'NEWS_AND_MAGAZINES': 2.81,
 'PARENTING': 0.72,
 'PERSONALIZATION': 3.49,
 'PHOTOGRAPHY': 2.79,
 'PRODUCTIVITY': 3.85,
 'SHOPPING': 2.01,
 'SOCIAL': 2.43,
 'SPORTS': 3.38,
 'TOOLS': 9.07,
 'TRAVEL_AND_LOCAL': 2.26,
 'VIDEO_PLAYERS': 1.83,
 'WEATHER': 0.84}

In [33]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [34]:
t = display_table(apple_final,11)
t

Games : 2257
Entertainment : 334
Photo & Video : 167
Social Networking : 143
Education : 132
Shopping : 121
Utilities : 109
Lifestyle : 94
Finance : 84
Sports : 79
Health & Fitness : 76
Music : 67
Book : 66
Productivity : 62
News : 58
Travel : 56
Food & Drink : 43
Weather : 31
Reference : 20
Navigation : 20
Business : 20
Catalogs : 9
Medical : 8


In [35]:
apple_freq

{'Book': 66,
 'Business': 20,
 'Catalogs': 9,
 'Education': 132,
 'Entertainment': 334,
 'Finance': 84,
 'Food & Drink': 43,
 'Games': 2257,
 'Health & Fitness': 76,
 'Lifestyle': 94,
 'Medical': 8,
 'Music': 67,
 'Navigation': 20,
 'News': 58,
 'Photo & Video': 167,
 'Productivity': 62,
 'Reference': 20,
 'Shopping': 121,
 'Social Networking': 143,
 'Sports': 79,
 'Travel': 56,
 'Utilities': 109,
 'Weather': 31}

#### Prime_Genre analysis


In [36]:
display_table(apple_final, 11)

Games : 2257
Entertainment : 334
Photo & Video : 167
Social Networking : 143
Education : 132
Shopping : 121
Utilities : 109
Lifestyle : 94
Finance : 84
Sports : 79
Health & Fitness : 76
Music : 67
Book : 66
Productivity : 62
News : 58
Travel : 56
Food & Drink : 43
Weather : 31
Reference : 20
Navigation : 20
Business : 20
Catalogs : 9
Medical : 8


In [37]:
display_table(google_final,1) #11

FAMILY : 1597
GAME : 768
TOOLS : 735
BUSINESS : 375
LIFESTYLE : 334
PRODUCTIVITY : 312
FINANCE : 311
PERSONALIZATION : 283
MEDICAL : 276
SPORTS : 274
COMMUNICATION : 244
HEALTH_AND_FITNESS : 232
NEWS_AND_MAGAZINES : 228
PHOTOGRAPHY : 226
SOCIAL : 197
TRAVEL_AND_LOCAL : 183
BOOKS_AND_REFERENCE : 181
SHOPPING : 163
VIDEO_PLAYERS : 148
MAPS_AND_NAVIGATION : 118
DATING : 112
FOOD_AND_DRINK : 96
LIBRARIES_AND_DEMO : 82
AUTO_AND_VEHICLES : 82
EDUCATION : 81
WEATHER : 68
EVENTS : 63
HOUSE_AND_HOME : 61
PARENTING : 58
ENTERTAINMENT : 57
ART_AND_DESIGN : 56
BEAUTY : 53
COMICS : 51


In [38]:
display_table(google_final,9)

Tools : 734
Entertainment : 500
Education : 448
Business : 375
Lifestyle : 333
Productivity : 312
Finance : 311
Personalization : 283
Sports : 277
Medical : 276
Action : 246
Communication : 244
Health & Fitness : 232
News & Magazines : 228
Photography : 226
Social : 197
Travel & Local : 182
Books & Reference : 181
Simulation : 175
Shopping : 163
Video Players & Editors : 148
Arcade : 147
Casual : 144
Maps & Navigation : 118
Dating : 112
Food & Drink : 96
Puzzle : 88
Racing : 83
Libraries & Demo : 82
Auto & Vehicles : 82
Role Playing : 80
Strategy : 75
Weather : 68
Events : 63
House & Home : 61
Adventure : 58
Beauty : 53
Art & Design : 52
Comics : 50
Parenting : 44
Card : 39
Casino : 38
Trivia : 37
Educational : 33
Educational;Education : 32
Board : 32
Education;Education : 26
Word : 21
Music : 17
Casual;Pretend Play : 17
Puzzle;Brain Games : 14
Racing;Action & Adventure : 11
Casual;Brain Games : 11
Entertainment;Music & Video : 10
Arcade;Action & Adventure : 10
Parenting;Education : 7


## Lets find out which genre is the most popular
For Apple store this will need to be done by the total number of user ratings
For Google play this will be done by the total number of downloads

In [39]:
# prime_genre = index 11
cnt_of_apps_to_genre = freq_table(apple_final,11)

In [40]:
for genre in cnt_of_apps_to_genre.keys():
    total = 0
    len_genre = 0
    for each in apple_final:
        if each[11] == genre:
            total += float(each[5])
            len_genre += 1
    avg = round(total / len_genre, 2)
    print(genre, ' - ', avg)

Business  -  6367.8
Catalogs  -  1779.56
Education  -  6266.33
Social Networking  -  53078.2
Medical  -  459.75
Entertainment  -  10822.96
Finance  -  13522.26
Health & Fitness  -  19952.32
Utilities  -  14010.1
Lifestyle  -  8978.31
Weather  -  47220.94
News  -  15892.72
Reference  -  67447.9
Music  -  56482.03
Games  -  18924.69
Food & Drink  -  20179.09
Shopping  -  18746.68
Book  -  8498.33
Photo & Video  -  27249.89
Travel  -  20216.02
Productivity  -  19053.89
Sports  -  20128.97
Navigation  -  25972.05


In [41]:
google_cols

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']

In [48]:
google_genres = freq_table(google_final,1)

In [50]:
for category in google_genres.keys():
    total = 0
    len_category = 0
    for each in google_final:
        if each[1] == category:
            total += float((each[5].replace('+','').replace(',','')))
            
            len_category += 1
    avg = round(total / len_category, 2)
    print(category, ' - ', avg)

GAME  -  7900871.68
BUSINESS  -  1118272.24
SHOPPING  -  3781218.31
COMMUNICATION  -  3729123.77
PRODUCTIVITY  -  5588555.49
EDUCATION  -  1152469.14
VIDEO_PLAYERS  -  14369808.92
SOCIAL  -  2745491.89
NEWS_AND_MAGAZINES  -  1254808.16
DATING  -  436205.87
PARENTING  -  542603.62
LIBRARIES_AND_DEMO  -  524339.15
HOUSE_AND_HOME  -  749220.67
COMICS  -  664042.16
FAMILY  -  2501813.21
TOOLS  -  7006861.87
MAPS_AND_NAVIGATION  -  2390345.59
FINANCE  -  800203.0
MEDICAL  -  90917.7
PHOTOGRAPHY  -  9554286.79
WEATHER  -  4342478.24
TRAVEL_AND_LOCAL  -  2435541.45
AUTO_AND_VEHICLES  -  647317.82
BOOKS_AND_REFERENCE  -  7792178.23
ENTERTAINMENT  -  8850175.44
EVENTS  -  253542.22
ART_AND_DESIGN  -  1843233.93
LIFESTYLE  -  1388516.25
SPORTS  -  3217265.27
HEALTH_AND_FITNESS  -  3924993.11
BEAUTY  -  513151.89
FOOD_AND_DRINK  -  1610820.32
PERSONALIZATION  -  3350656.85
